# TPALTDAtaMart

Membres du groupe :
- **TOUNA ETOUNDI ALEXIS JANVIER**

- **BADJENE HONORE**








 ## TP1
* Ecrivez deux programme Python dans le dossier src/data *
- Le premier récupère tous les dataset de 2018 à 2023
- Le deuxième récupère le dernier mois
- Les données seront téléchargées et stockées dans minio

***Question 1 :***
 j'utilise principalement 3 bibliotheques pour cette question 
- *request* pour envoiyuer des requettes http 
- *tqdm* il me pertmet d'affichier une barre de progression pour que le script ne soit pas muets 
- *datetime* pour les calculs de dates et aussi pour obtenir la date en temps reels

In [ ]:
import requests
from tqdm import tqdm
from datetime import datetime, timedelta

def download_file(url, local_filename):

    #  j'envoie une requête GET à l'URL
    response = requests.get(url, stream=True)
    response.raise_for_status()  # je vérifie si la requête a réussi

    # je Récupère la taille totale du fichier
    total_size = int(response.headers.get('content-length', 0))

    # j'ouvre un fichier en mode binaire pour l'écriture
    with open(local_filename, 'wb') as file:
        # on Utilise tqdm pour afficher la barre de progression donne un rendu visuel plus facile a apprecier 
        for chunk in tqdm(response.iter_content(chunk_size=8192), total=total_size // 8192, unit='KB'):
            file.write(chunk)

def generate_date_range(start_date, end_date):
   """ on genere l'intervelle de date pour lîteration """
    
    start = datetime.strptime(start_date, "%Y-%m")
    end = datetime.strptime(end_date, "%Y-%m")
    date_list = []
    while start <= end:
        date_list.append(start.strftime("%Y-%m"))
        start += timedelta(days=31)  # on passe au mois suivant
        start = start.replace(day=1)  # on Fixe le jour au 1er du mois
    return date_list

def download_files_in_date_range(base_url, start_date, end_date):
  
    dates = generate_date_range(start_date, end_date)
    for date in dates:
        url = f"{base_url}_{date}.parquet"
        local_filename = f"yellow_tripdata_{date}.parquet"
        print(f"Téléchargement de {url}...")
        download_file(url, local_filename)
        print(f"{local_filename} téléchargé avec succès.")

# URL de base pour les fichiers à télécharger
base_url = "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata"

# Dates de début et de fin
start_date = "2018-01"
end_date = "2023-01"

# on appelle la fonction pour télécharger les fichiers dans l'intervalle de dates
download_files_in_date_range(base_url, start_date, end_date)



![Texte alternatif](P1.png)

* On peut voir la progression de chaque telechargement par fichier 

***Question 2 :***
Ici il est question de telecharger le dernier mois uniquemnet 
il faut:
- tenir compte de la date du jour et recuperer le mois actuel
- verifier de la disponibilité des données du mois precedent et les telecharger
- Si les données ne sont pas disponibles reculer de nouveau d'un mois puis verifier à nouveau



In [ ]:
def download_previous_month_data():
    # on récupère la date du jour
    today = datetime.datetime.today()
    
    while True:
        # on écupère l'année et le mois actuels
        year = today.year
        month = today.month
        
        # on Formate le mois pour qu'il soit toujours sur deux chiffres
        formatted_month = f"{month:02d}"
        
        # on construit l'URL du fichier à télécharger
        url = f"https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_{year}-{formatted_month}.parquet"
        
        # Nom du fichier local où le fichier sera sauvegardé
        file_name = f"yellow_tripdata_{year}-{formatted_month}.parquet"
        
        # Télécharger le fichier
        response = requests.get(url)
        
        # On vérifie si le fichier est disponible
        if response.status_code == 200:
            with open(file_name, 'wb') as file:
                file.write(response.content)
            print(f"Fichier téléchargé et sauvegardé sous le nom {file_name}")
            break  # on Sort de la boucle si le fichier est téléchargé avec succès
        else:
            print(f"Le fichier pour le mois {formatted_month}-{year} n'est pas disponible.")
            
            # on recule d'un mois
            today = today - datetime.timedelta(days=today.day)  # Recule au premier jour du mois précédent
            if today.month == 12:
                today = today.replace(year=today.year - 1, month=12)
            else:
                today = today.replace(month=today.month - 1)
    
    print("Fin du téléchargement.")

# Appele la fonction
download_previous_month_data()



![Texte alternatif](P3.png)

***Question 3 : Combiner les fichier parquet et les stocker***

- Fusion et le stockage dans minIO

In [ ]:
import pandas as pd
import os

# repertoire contenant les parquets
data_dir = "C:\\Users\\Aorus 2022\\Desktop\\Python Scripts\\RAW"

# Initialise la liste vide pour stocker le DataFrames
all_data = []

# boucle sur chaque fichier du repertoire
for filename in os.listdir(data_dir):
  if filename.endswith(".parquet"):
    # Read the parquet file into a DataFrame
    df = pd.read_parquet(os.path.join(data_dir, filename))
    
    # Ajouter le dataframe a la liste
    all_data.append(df)

# Concatène les Dataframes en un seul
combined_df = pd.concat(all_data, ignore_index=True)


output_file = "combined_yellow_data.parquet" 
combined_df.to_parquet(output_file)

print(f"Fichiers combinés avec succès {output_file}")



 ## TP2

- Récupérez les données que vous avez téléchargé en local. 
- Fusionnez les données pour avoir une unique table. 
- Chargez le résultat vers votre base de données

![Texte alternatif](P4.png)

Pour le chargement dans la base de donnée j'ai utilisé une bdd MySql

 ## TP3

### Nombre total de trajets 
Ce KPI mesure le nombre total de trajets en taxi effectués pendant une période spécifique. Il fournit un volume global d'activité et peut être utilisé pour suivre la croissance ou la baisse de la demande.

Insight: Un nombre élevé de trajets peut indiquer une période chargée, tandis qu'un nombre faible peut suggérer une baisse de la demande.

### Durée moyenne des trajets
Ce KPI calcule la durée moyenne de chaque trajet en taxi. Il aide à comprendre l'efficacité du service et à identifier les opportunités de réduire les temps de trajet.

Insight: Une durée moyenne de trajet plus courte peut indiquer des routes plus efficaces ou une meilleure gestion du trafic, tandis qu'une durée plus longue peut suggérer des congestions ou des routes inefficaces.

### Lieux de prise en charge les plus populaires
Ce KPI identifie les lieux où les passagers sont les plus susceptibles de héler un taxi. Il aide à optimiser le déploiement des taxis et l'allocation des ressources.
Insight: Connaître les lieux de prise en charge les plus populaires peut aider les compagnies de taxi à positionner stratégiquement leurs véhicules, réduisant les temps d'attente et augmentant les chances d'obtenir une course.

### Lieux de dépose les plus populaires
Ce KPI identifie les lieux où les passagers sont les plus susceptibles d'être déposés. Il aide à comprendre le comportement des passagers et à optimiser les routes des taxis.
Insight: Connaître les lieux de dépose les plus populaires peut aider les compagnies de taxi à optimiser leurs routes, réduisant les temps de trajet et augmentant les chances d'obtenir une course.

### Tarif moyen 
Ce KPI mesure le montant moyen payé par les passagers pour chaque trajet. Il aide à comprendre les tendances de revenu et à identifier les opportunités d'augmenter les gains.

Insight: Un tarif moyen élevé peut indiquer un service premium ou une forte demande, tandis qu'un tarif moyen faible peut suggérer un marché compétitif ou une faible demande.

### 5 modes de paiement les plus courants 

Ce KPI identifie les modes de paiement les plus couramment utilisés par les passagers. Il aide à comprendre le comportement des passagers et à optimiser le traitement des paiements.

Insight: Connaître les modes de paiement les plus courants peut aider les compagnies de taxi à optimiser leur traitement des paiements, réduisant les frais de transaction et augmentant les revenus.

### Heures de pointe
Ce KPI identifie les heures les plus chargées de la journée où les passagers sont les plus susceptibles de héler un taxi. Il aide à optimiser le déploiement des taxis et l'allocation des ressources.

Insight: Connaître les heures de pointe peut aider les compagnies de taxi à positionner stratégiquement leurs véhicules, réduisant les temps d'attente et augmentant les chances d'obtenir une course pendant les périodes chargées.

## Le modèle en flocon 
### DimensionTemporelle
Cette dimension stocke les informations temporelles liées aux courses de taxi.

pickup_datetime : La date et l'heure du début de la course.
dropoff_datetime : La date et l'heure de la fin de la course.
### DimensionLieu
Cette dimension contient des informations sur les emplacements géographiques liés aux courses de taxi.

pulocationid : L'identifiant du lieu de prise en charge.
dolocationid : L'identifiant du lieu de dépose.
### DimensionFournisseur
Cette dimension stocke des informations sur les fournisseurs de services de taxi.

vendorid : L'identifiant du fournisseur de taxi.
### DimensionTarification
Cette dimension stocke des informations sur les différentes tarifications utilisées.

ratecodeid : L'identifiant de la tarification utilisée pour la course.
### DimensionPaiement
Cette dimension stocke des informations sur les types de paiement.

payment_type : Le type de paiement utilisé pour la course.
### FactTable
La table de faits stocke des mesures (quantités numériques) associées à chaque course, ainsi que des clés étrangères vers les tables de dimensions correspondantes.

- pickup_datetime : Clé étrangère vers la table DimensionTemporelle.
- dropoff_datetime : Clé étrangère vers la table DimensionTemporelle.
- pulocationid : Clé étrangère vers la table DimensionLieu.
- dolocationid : Clé étrangère vers la table DimensionLieu.
- vendorid : Clé étrangère vers la table DimensionFournisseur.
- ratecodeid : Clé étrangère vers la table DimensionTarification.
- payment_type : Clé étrangère vers la table DimensionPaiement.
- trip_distance : La distance parcourue lors de la course.
- fare_amount : Le montant de la course.
- extra : Suppléments éventuels.
- mta_tax : Taxe MTA.
- tip_amount : Montant du pourboire.
- tolls_amount : Montant des péages.
- improvement_surcharge : Supplément d'amélioration.
- total_amount : Montant total de la course.
- congestion_surcharge : Supplément pour congestion.
- airport_fee : Frais d'aéroport.

# TP4

Pour la realisation des visuels et des DASHBORD j'ai utilisé Tableau

![Texte alternatif](P5.png)

In [ ]:
Après chargement des sources de données on peut commencer a visualiser les KPI.
Du fait des limitates de performance de mon systeme 

#### Du a une erreur et un manque de performance je n'ai pas reussi à formater le fichier recombiné 

en effet les operations de conversion sur le format des dates devais etre effectué
* tpep_pickup_datetime" & tpep_dropoff_datetime" Etaient en nombre de seconde depuis 1 jan 1970 il fallait donc effectuer une convertion pour obtenir des données exploitable dans tableau

![Texte alternatif](P7.png)

notez l'erreur d'allocation memoire en bas a droite

##  Realisation de quelques visuels

### Durée moyenne de trajet par l'heure
- on peut noter les pics de traffic en debut d'apres midi


![Texte alternatif](P8.png)

## Tarif moyen d'une course et son evolution entre 2008 et 2009
![Texte alternatif](P6.png)

## CREATION DES DASHBORD
- Il suffit d'ajouter les feuilles qui nous interessent a notre dashbord nouvellenebt cree et lel tour est joué

## Distance moyenne par course
![Texte alternatif](P9.png)

## DASHBORD CONTENANT LES KPI visualisé plus haut
![Texte alternatif](P11.png)